# Flag Additions Optimize

Optimizes file removing escape characters and decreasing the flag size

In [8]:
import requests
import json
import urllib

## Open the file

In [9]:
flagsPath = "./in/flags_additions.json"

with open(flagsPath, "r", encoding="utf-8") as json_file:
    flags = json.load(json_file)

## Unescape characters in flags links

In [10]:

for country in flags:
    regions = flags[country]['regions']
    for region in regions.values():
        flag = region['flag']
        region['flag'] = urllib.parse.unquote(flag)


## Reduce the flag size

### Undo previous png optimization

In [4]:
for country in flags.values():
    for flagData in country["regions"].values():
        flag: str = flagData["flag"]
        if "upload.wikimedia.org" not in flag:
            continue

        textToMatch = ".svg"
        constainsSvg = flag.find(textToMatch)
        if constainsSvg != -1:
            flagData["flag"] = flag[:constainsSvg + len(textToMatch)]


In [11]:
image_url = "https://upload.wikimedia.org/wikipedia/commons/4/43/Flag_of_Southwest_Papua.svg"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"}


def urlContentSize(url):
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        file_size_bytes = len(response.content)
        return file_size_bytes / 1024
    else:
        print(
            f"Failed to fetch the image. Status code: {response.status_code}")


In [12]:
maxSvgSize = 100

for country in flags.values():
    for flagData in country["regions"].values():
        flag: str = flagData["flag"]
        extension = flag.split(".")[-1]
        if extension != "svg" or "upload.wikimedia.org" not in flag:
            continue
        size = urlContentSize(flag)
        if size > maxSvgSize:
            print("Flag:", flag, "Size:", size, "KB")

            sections = flag.split("/")
            commons_index = sections.index('commons')
            sections.insert(commons_index + 1, 'thumb')

            filename = sections[-1]
            addedFilename = "/128px-" + filename + ".png"
            sections.append(addedFilename)

            newFlag = '/'.join(sections[:])
            # print("New flag:", newFlag)
            # print("New flag:", newFlag, "Size:", urlContentSize(newFlag), "KB")
            flagData["flag"] = newFlag


Flag: https://upload.wikimedia.org/wikipedia/commons/c/c3/Coat_of_arms_of_Vidzeme.svg Size: 126.0595703125 KB
Flag: https://upload.wikimedia.org/wikipedia/commons/b/b8/Flag_of_Zaporizhia_Oblast.svg Size: 149.20703125 KB


In [13]:
json_data = json.dumps(
    flags, indent=4, ensure_ascii=False).encode('utf8').decode()

with open(flagsPath, "w") as json_file:
    json_file.write(json_data)

print("JSON data has been saved to", flagsPath)


JSON data has been saved to ./in/flags_additions.json
